In [1]:
import ipywidgets as widgets
from IPython.display import display
from threading import Thread
from queue import Queue

In [2]:
messages=Queue()
recordings=Queue()



record_button=widgets.Button(
    description="Record",
    disabled=False,
    button_style="success", 
    icon="microphone"
)


stop_button=widgets.Button(
    description="Stop",
    disabled=False,
    button_style="warning",
    icon="stop"

)
output=widgets.Output()

def start_recording(data):
    messages.put(True)
    with output:
        display("Starting...")
        # record=Thread(target=record_microphone)
        # record.start()

        # transcribe=Thread(target=speech_recognition,args=(output,))
        # transcribe.start()

def stop_recording(data):
    with output:
        messages.get()
        display("Stopped...")

record_button.on_click(start_recording)
stop_button.on_click(stop_recording)

display(record_button,stop_button,output)

Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle())

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle())

Output()

In [5]:
import pyaudio
p=pyaudio.PyAudio()

for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))

p.terminate()

ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4528:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4528:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4528:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5007:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2495:(snd_pcm_open_noupdate) Unknown PCM sysdefault
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4528:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4528:(_snd_config_evaluate) function snd_func_concat returned error

In [ ]:
CHANNELS=1
FRAME_RATE=16000
RECORD_SECONDS=20
AUDIO_FORMAT=pyaudio.paInt16
SAMPLE_SIZE=2

def record_microphone(chunk=1024):
    p=pyaudio.PyAudio()
    stream=p.open(format=AUDIO_FORMAT,
                  channels=CHANNELS,
                  rate=FRAME_RATE,
                  input=True,
                  input_device_index=2,
                  frames_per_buffer=chunk
                  )
    
    frames=[]

    while not messages.empty():
        data=stream.read(chunk)
        frames.append(data)

        if len(frames)>=(FRAME_RATE*RECORD_SECONDS)/chunk:
            recordings.put(frames.copy())
            frames=[]
        
    
    stream.stop_stream()
    stream.close()
    p.terminate()



In [6]:
import subprocess
import json
from vosk import Model, KaldiRecognizer


In [ ]:
model=Model(mode_name="vosk-model-en-us-0.22")
rec=KaldiRecognizer(model,FRAME_RATE)
rec.SetWords(True)

def speech_recognition(output):
    while not messages.empty():
        frames=recordings.get()

        rec.AcceptWaveform(b''.join(frames))
        result=rec.Result()
        text=json.loads(result)["text"]

        cased=subprocess.check_output("python recasepunc/recasepunc.py predict recasepunc/checkpoint",shell=True,text=True,input=text)
        output.append_stdout(cased)